!pip install numpy tqdm scikit-learn tensorflow keras

In [1]:
# Import packages needed
import os
import torch
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from keras.layers import Lambda, Dense
from keras import Model
from keras.models import load_model, Sequential
from keras.ops import cast, maximum, square
from keras.metrics import binary_accuracy
from keras.applications import ResNet50
from keras import Input

from tqdm import tqdm
from sklearn.model_selection import train_test_split

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


2024-12-10 06:48:26.592173: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733813306.661981   67935 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733813306.684242   67935 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-10 06:48:26.843226: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


cuda


In [2]:
# Function for reading all the image from the dataset folder
def get_data():
    # read all the folders
    data_path = "../../dataset/Musinsa_dataset"
    folders = os.listdir(data_path)

    # read all the images inside the folders
    style2index = []
    images = []
    labels = []
    for i in range(len(folders)):
        folder = folders[i]
        folder_path = f"{data_path}/{folder}"

        if not os.path.isdir(folder_path):
            continue

        files = os.listdir(folder_path)
        print(folder)

        count = 0

        for file in tqdm(files):
            try:
                # read the image
                image = cv2.imread(f"{folder_path}/{file}")
                image = cv2.resize(image, (224, 224))
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = image.astype(np.float32) / 255.0

                images.append(image)
                labels.append(i)
                count += 1
                if count >= 2000:
                    break

            except Exception as e:
                print(f"Error reading image {file}: {e}")
        style2index.append({folder: i})

    return images, labels, style2index

In [3]:
image, labels, style2index = get_data()
print(len(image), len(labels))
print(style2index)

Classic


100%|██████████| 1642/1642 [00:02<00:00, 804.44it/s]


Chic


100%|██████████| 1780/1780 [00:02<00:00, 837.75it/s]


Cityboy


...

Romantic


100%|██████████| 1795/1795 [00:02<00:00, 815.39it/s]


Girlish


100%|██████████| 1783/1783 [00:02<00:00, 819.41it/s]

23348 23348
[{'Classic': 1}, {'Chic': 2}, {'Cityboy': 3}, {'Casual': 4}, {'Minimal': 5}, {'Preppy': 6}, {'Workwear': 7}, {'Retro': 8}, {'Street': 9}, {'Gorpcore': 10}, {'Sporty': 11}, {'Romantic': 12}, {'Girlish': 13}]


In [4]:
X_train, X_test, y_train, y_test = train_test_split(image, labels, test_size=0.2, random_state=42)

print(f"train set size: {len(X_train)}, {len(y_train)}")
print(f"test set size: {len(X_test)}, {len(y_test)}")

train set size: 18678, 18678
test set size: 4670, 4670


In [5]:
X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(y_train)
Y_test = np.array(y_test)

print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(18678, 224, 224, 3) (4670, 224, 224, 3) (18678,) (4670,)


In [6]:
def generate_pair(X, y):
    """
        For contrastive learning, we need the dataset in pair.
        There should exist 
        Input: X(image), y(label)
        Output: X_pairs(image pair), y_pairs(label pair)
    """
    X = np.array(X)
    y = np.array(y)
    
    X_pairs = []
    y_pairs = []

    for i in range(len(X)):
        digit = y[i]

        positive_digit_index = np.random.choice(np.where(y == digit)[0])
        X_pairs.append([X[i], X[positive_digit_index]])
        y_pairs.append([0])

        negative_digit_index = np.random.choice(np.where(y!=digit)[0])
        X_pairs.append([X[i], X[negative_digit_index]])
        y_pairs.append([1])

    indices = np.arange(len(X_pairs))
    np.random.shuffle(indices)

    return np.array(X_pairs)[indices], np.array(y_pairs)[indices]

In [7]:
with tf.device('/GPU:0'):
    X_train_pairs, Y_train_pairs = generate_pair(X_train, y_train)
    X_test_pairs, Y_test_pairs = generate_pair(X_test, y_test)

    print("X_train_pairs shape: ", X_train_pairs.shape)
    print("X_test_pairs shape: ", X_test_pairs.shape)

I0000 00:00:1733813342.380022   67935 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22456 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:61:00.0, compute capability: 8.6


X_train_pairs shape:  (37356, 2, 224, 224, 3)
X_test_pairs shape:  (9340, 2, 224, 224, 3)


# Model

### Check GPU

In [8]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("Physical GPUs: ", tf.config.list_physical_devices('GPU'))

Num GPUs Available:  1
Physical GPUs:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [9]:
# tf.debugging.set_log_device_placement(True)


### ResNet based model

Using feature extraction part of pretrained ResNet model, one fc layer will be added on it.

In [10]:
# Use backbone of ResNet50

with tf.device('/GPU:0'):
    input1 = Input(shape=(224,224,3,))
    input2 = Input(shape=(224,224,3,))

    base_model = ResNet50(weights="imagenet", include_top=True)

    network = Sequential(
        [
            Input(shape=(224, 224, 3)),
            base_model,
            Dense(128, activation=None)
        ]
    )


In [11]:
with tf.device('/GPU:0'):
    twin1 = network(input1)
    twin2 = network(input2)

In [12]:
def cosine_distance(twins):
    twin1_output, twin2_output = twins
    twin1_norm = tf.linalg.l2_normalize(twin1_output, axis=1)
    twin2_norm = tf.linalg.l2_normalize(twin2_output, axis=1)

    cosine_similarity = twin1_norm * twin2_norm  # Element-wise multiplication
    cosine_similarity = tf.reduce_sum(cosine_similarity, axis=1, keepdims=True)

    return (1 - cosine_similarity)

In [13]:
with tf.device('/GPU:0'):
    distance = Lambda(cosine_distance)([twin1, twin2])
    model = Model(inputs=[input1, input2], outputs=distance)

In [14]:
def contrastive_loss(y, d):
    """
    Compute the contrastive loss introduced by Yann LeCun et al. in the paper
    "Dimensionality Reduction by Learning an Invariant Mapping."
    """

    margin = 1
    y = cast(y, d.dtype)

    loss = (1 - y) / 2 * square(d) + y / 2 * square(maximum(0.0, margin - d))

    return loss


In [15]:
with tf.device('/GPU:0'):
    model.compile(loss=contrastive_loss, optimizer="adam", metrics=[binary_accuracy])


In [16]:
with tf.device('/GPU:0'):
    tf.debugging.set_log_device_placement(True)
    history = model.fit(
        x=[X_train_pairs[:, 0], X_train_pairs[:, 1]],
        y=Y_train_pairs[:],
        validation_data=([X_test_pairs[:, 0], X_test_pairs[:, 1]], Y_test_pairs[:]),
        batch_size=8,
        epochs=5,
    )
    tf.debugging.set_log_device_placement(False)

2024-12-10 06:49:28.550243: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 22492495872 exceeds 10% of free system memory.


: 

In [ ]:
with tf.device('/GPU:0'):
    plt.plot(history.history["loss"])
    plt.plot(history.history["val_loss"])
    plt.title("Training and Validation Loss")
    plt.ylabel("loss")
    plt.xlabel("epoch")
    plt.legend(["train", "val"], loc="upper right")
    plt.show()

In [ ]:
with tf.device('/GPU:0'):
    predictions = model.predict([X_test_pairs[:, 0], X_test_pairs[:, 1]]) >= 0.5

In [ ]:
with tf.device('/GPU:0'):
    print(model.layers)
    print(model.layers[2].input)

In [ ]:
def load_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (224, 224))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image.astype(np.float32) / 255.0

    return image

In [ ]:
with tf.device('/GPU:0'):
    embedding_model = model.layers[2]
    print(embedding_model)

    image_path = "../../dataset/Musinsa_dataset/Cityboy/snap_card_1305697549538040667.jpg"
    image = load_image(image_path)
    embedding = embedding_model.predict(image.reshape(1, 224, 224, 3))

    print(embedding.shape)


In [ ]:
with tf.device('/GPU:0'):
    # Save the model
    embedding_model = Model(inputs=input1, outputs=twin1)

    embedding_model.save("embedding_model.h5")

In [ ]:
with tf.device('/GPU:0'):
    # Load the model
    loaded_model = load_model("embedding_model.h5")

    image_path = "../../dataset/Musinsa_dataset/Cityboy/snap_card_1305697549538040667.jpg"
    image = load_image(image_path)

    embedding = loaded_model.predict(image.reshape(1, 224, 224, 3))

    print("Generated embedding: ", embedding)